<a href="https://colab.research.google.com/github/aimyapp/mapleEventNotice/blob/main/mapleEventNotice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title System Setup
import subprocess

subprocess.run("pip install selenium", shell=True, check=True)
subprocess.run("sudo apt -y update", shell=True, check=True)
subprocess.run("apt-get install libvulkan1", shell=True, check=True)
subprocess.run("sudo apt -f install", shell=True, check=True)
subprocess.run("sudo apt install -y wget curl unzip", shell=True, check=True)
subprocess.run("wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb", shell=True, check=True)
subprocess.run("dpkg -i libu2f-udev_1.1.4-1_all.deb", shell=True, check=True)
subprocess.run("wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb", shell=True, check=True)
subprocess.run("dpkg -i google-chrome-stable_current_amd64.deb", shell=True, check=True)
subprocess.run("wget -N https://storage.googleapis.com/chrome-for-testing-public/130.0.6723.116/linux64/chromedriver-linux64.zip -P /tmp/", shell=True, check=True)
subprocess.run("unzip -o /tmp/chromedriver-linux64.zip -d /tmp/", shell=True, check=True)
subprocess.run("chmod +x /tmp/chromedriver-linux64/chromedriver", shell=True, check=True)
subprocess.run("mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver", shell=True, check=True)

CompletedProcess(args='mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver', returncode=0)

In [2]:
#@title Confirm Version
!google-chrome --version
!chromedriver --version

Google Chrome 131.0.6778.85 
ChromeDriver 130.0.6723.116 (6ac35f94ae3d01152cf1946c896b0678e48f8ec4-refs/branch-heads/6723@{#1764})


In [3]:
#@title Main Feature
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep
from google.colab import userdata

# Chromeオプションの設定
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--remote-debugging-port=9222')

# ChromeDriverのパスを設定
chrome_service = Service('/usr/local/bin/chromedriver')

# ブラウザを起動
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# 指定されたURLを開く
driver.get('https://maplestory.nexon.co.jp/notice/list/event/')

# class="card-list"の要素を取得
card_list = driver.find_element(By.CLASS_NAME, 'card-list')

# ul配下のすべてのli要素を取得
list_items = card_list.find_elements(By.TAG_NAME, 'li')

access_token = userdata.get('ACCESS_TOKEN')
user_id = userdata.get('USER_ID')

# 各li要素の情報を収集
messages = []
for item in list_items:
    link = item.find_element(By.TAG_NAME, 'a')
    href = link.get_attribute('href')
    title = item.find_element(By.CLASS_NAME, 'title').text
    date = item.find_element(By.CLASS_NAME, 'date').text
    print(f"Title: {title}\nDate: {date}\nLink: {href}\n")
    messages.append(f"{title}\n{date}\n{href}\n")

# LINE Messaging APIを使って通知を送信
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# お知らせ開始の挨拶
data = {
    "to": user_id,
    "messages": [
        {
          "type":"text",
          "text":"開催中のイベントぷるー!!!!"
        }
    ]
}
response = requests.post("https://api.line.me/v2/bot/message/push", headers=headers, json=data)

# 開催中のイベントを5件ごとに一つにまとめて送信
for i in range(0, len(messages), 5):
    batch = messages[i:i+5]
    combined_message = "\n".join(batch)
    notice_data = {
        "to": user_id,
        "messages": [
            {
                "type": "text",
                "text": combined_message
            }
        ]
    }
    response = requests.post("https://api.line.me/v2/bot/message/push", headers=headers, json=notice_data)
    print(response.status_code)
    print(response.text)

# ブラウザを終了
driver.quit()


Title: ラッキードローキャンディー！
Date: 11/20(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=54e22dd67a2d41d08f0f7b3ae47f9b32&id=event

Title: 走れ！ピッピープップーバンパーカー
Date: 11/13(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=ff18ebedefa247328dcdef7bb509e8e3&id=event

Title: 空からオモチャのブロックが降ってきたら？
Date: 11/13(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=7e2b1e85ed4f41adaae2bfe55ce46f91&id=event

Title: ドキドキクレーンゲーム(11/6更新)
Date: 11/6(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=8d804dcd205a446d970458dace232f98&id=event

Title: 襲来！ブロックビシャスプラント
Date: 11/6(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=3bcd1bc174bb4d77b0807311aa1e49d6&id=event

Title: 夜のサーカス団
Date: 10/30(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=db3ee225766645e1abce9e7a43a47a0e&id=event

Title: タイムトラベルチケット
Date: 10/30(水)~11/26(火)
Link: https://maplestory.nexon.co.jp/notice/view/?alias=817862dc30084fce99f2044ef